In [2]:
import pandas as pd
import os
from tqdm import tqdm
from utils import *
full_data_folder = '/Users/junyi/Work/RR/rr-measure-dataset/journal-full-text'
meta_data_folder = '/Users/junyi/Work/RR/rr-measure-dataset/journal-meta'

In [3]:
data = pd.read_csv(os.path.join(meta_data_folder, 'full-meta-dataset.csv'))
data.columns

Index(['title', 'doi', 'volume', 'date', 'year', 'month', 'abstract', 'issn',
       'journal_name', 'unique_id', 'is_github', 'num_of_github_urls'],
      dtype='object')

In [ ]:
data['is_availablity_statement'] = 0
for i in range(len(data)):
# # In case for the mini test to debug the code
# for i in tqdm(range(1000)):
    journal_path = os.path.join(full_data_folder, data['issn'][i])  # Path to the journal folder
    paper_path = os.path.join(journal_path, data['unique_id'][i] + '.xml')  # Path to the paper folder
    sections = extract_sections_and_text_from_xml(paper_path)
    reorganized_sections = postprocess_sections(sections)
    for section in reorganized_sections:
        section_title = section['title']
        if 'data availability' in section_title.lower():
            data.loc[i, 'is_availablity_statement'] = 1
data.to_csv(os.path.join(meta_data_folder, 'full-meta-dataset.csv'), index=False)

In [20]:
data['is_data_mentioned_in_section_title'] = 0
for i in range(len(data)):
# # In case for the mini test to debug the code
# for i in tqdm(range(1000)):
    journal_path = os.path.join(full_data_folder, data['issn'][i])  # Path to the journal folder
    paper_path = os.path.join(journal_path, data['unique_id'][i] + '.xml')  # Path to the paper folder
    sections = extract_sections_and_text_from_xml(paper_path)
    reorganized_sections = postprocess_sections(sections)
    label = 0
    for section in reorganized_sections:
        section_title = section['title']
        if 'data' in section_title.lower():
            label = 1
        for subsection in section['subsections']:
            subsection_title = subsection['title']
            if 'data' in subsection_title.lower():
                label = 1
            for subsubsection in subsection['subsubsections']:
                subsubsection_title = subsubsection['title']
                if 'data' in subsubsection_title.lower():
                    label = 1
    data.loc[i, 'is_data_mentioned_in_section_title'] = label
data.to_csv(os.path.join(meta_data_folder, 'full-meta-dataset.csv'), index=False)

In [31]:
data['is_experiment_mentioned_in_section_title'] = 0
for i in range(len(data)):
# # In case for the mini test to debug the code
# for i in tqdm(range(1000)):
    journal_path = os.path.join(full_data_folder, data['issn'][i])  # Path to the journal folder
    paper_path = os.path.join(journal_path, data['unique_id'][i] + '.xml')  # Path to the paper folder
    sections = extract_sections_and_text_from_xml(paper_path)
    reorganized_sections = postprocess_sections(sections)
    label = 0
    for section in reorganized_sections:
        section_title = section['title']
        if 'experiment' in section_title.lower():
            label = 1
        for subsection in section['subsections']:
            subsection_title = subsection['title']
            if 'experiment' in subsection_title.lower():
                label = 1
            for subsubsection in subsection['subsubsections']:
                subsubsection_title = subsubsection['title']
                if 'experiment' in subsubsection_title.lower():
                    label = 1
    data.loc[i, 'is_experiment_mentioned_in_section_title'] = label
data.to_csv(os.path.join(meta_data_folder, 'full-meta-dataset.csv'), index=False)

In [ ]:
data['is_link_in_avaiablity_statement'] = 0
data['num_of_links_in_avaiablity_statement'] = 0
url_pattern = r'(https?://\S+|www\.\S+)'  # URL pattern
# # In case for the mini test to debug the code
url_data = []
for i in range(len(data)):
# for i in range(1000):
    journal_path = os.path.join(full_data_folder, data['issn'][i])  # Path to the journal folder
    paper_path = os.path.join(journal_path, data['unique_id'][i] + '.xml')  # Path to the paper folder
    sections = extract_sections_and_text_from_xml(paper_path)
    reorganized_sections = postprocess_sections(sections)
    for section in reorganized_sections:
        section_title = section['title']
        if 'data availability' in section_title.lower():
            if 'http' in section['text']:
                data.loc[i, 'is_link_in_avaiablity_statement'] = 1
                url = re.findall(url_pattern, section['text'])
                unique_url = list(set(url))
                if url:
                    print(url)
                url_data.append({
                'issn': data['issn'][i],
                'unique_id': data['unique_idf'][i],
                'title': data['title'][i],
                'url': unique_url
                })
                data.loc[i, 'num_of_links_in_avaiablity_statement'] = len(unique_url)
data.to_csv(os.path.join(meta_data_folder, 'full-meta-dataset.csv'), index=False)
save_json(url_data, os.path.join(meta_data_folder, 'url_data.json'))

['http://doi.org/10.17605/OSF.IO/DE8KM.']
['https://doi.org/10.7910/DVN/VGMDPC).']
['https://covid19.apple.com/mobility,', 'https://www.google.com/covid19/mobility,', 'https://covid19.saglik.gov.tr.']
['https://doi.org/10.17632/tfkgpmgf9t.1.']
['https://dx.doi.org/10.17632/5gy6s8trbm.2']
['https://github.com/Lilyhanig/transit_covid_precautions.']
['https://doi.org/10.6084/m9.figshare.20311812.v2']
['https://doi.org/10.6084/m9.figshare.24231706.']
['https://doi.org/10.25338/B8035D']
['https://doi.org/10.4121/e4324d0c-2a82-4f03-8cbb-64dd207cd522.']
['https://osf.io/sw8k2/.']
['https://data.gesis.org/sharing/#!Detail/10.7802/2155.']
['https://zenodo.org/doi/10.5281/zenodo.10778028']
['https://doi.org/10.17605/OSF.IO/X7AER).']
['https://www.sciencedirect.com/science/article/pii/S2352340923001452']
['https://datadryad.org/stash/dataset/doi:10.5061/dryad.rv15dv4f0']
['https://doi.org/10.4121/a59fcc0f-4603-490a-b0b8-8b2287141e30.', 'https://repo.bayesfusion.com/network/permalink?net=Small+BNs